# Importing Libraries

In [1]:
import pandas as pd
from os import listdir
from os.path import isfile, join

# File Dimensions

In [10]:
data_path = 'Data/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
log_i = 0
file_i = 0

for file in onlyfiles:
    if file[:3] == 'LOG':
        log_i += 1
        print('log file ', log_i, ': ', file)
    else:
        file_i += 1
        print('non-log file ', file_i, ': ', file)
    print('it has: ', len(pd.read_csv(data_path + file, sep = ';', error_bad_lines = False)), 'lines')

non-log file  1 :  TABLE_CONVERSION.csv
it has:  580404 lines
non-log file  2 :  CLIENTS.csv
it has:  3125800 lines
log file  1 :  LOG_WEB_201710.csv
it has:  3301363 lines
log file  2 :  LOG_WEB_201711.csv
it has:  5055599 lines
non-log file  3 :  CMD.csv
it has:  7323227 lines
log file  3 :  LOG_WEB_201712.csv
it has:  4535451 lines
log file  4 :  LOG_WEB_201708.csv
it has:  2189118 lines
log file  5 :  LOG_WEB_201709.csv
it has:  2874217 lines
log file  6 :  LOG_WEB_201801.csv


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


it has:  8450825 lines
log file  7 :  LOG_WEB_201802.csv
it has:  6063362 lines
log file  8 :  LOG_WEB_201803.csv
it has:  2739719 lines


# LOG Exploration

In [32]:
data_201708 = pd.read_csv(data_path + 'LOG_WEB_201708.csv', sep=';')

In [25]:
data_201708.head()

,Unnamed: 0,VISITOR_ID,CONNECTED_SESSION,ID_SESSION,SESSION_START_DATE,EVENT_DATE,PAGES,GLOBAL_SOURCES,DEVICE_TYPE,DEVICE_MODEL,LOADINGS
0,0,381225,OUI,1,2017-08-08 00:02:06,2017-08-08 00:02:06,accueil,Liens Sponsorisés,Smartphone,NaN,1
1,1,563053,OUI,13,2017-08-08 00:01:32,2017-08-08 00:03:27,accueil,Accès Direct,Ordinateur,NaN,1
2,2,563053,OUI,13,2017-08-08 00:01:32,2017-08-08 00:22:04,accueil,Accès Direct,Ordinateur,NaN,1
3,3,706588,OUI,23,2017-08-08 00:01:29,2017-08-08 00:01:32,accueil,Accès Direct,Ordinateur,NaN,1
4,4,0,NON,27,2017-08-08 00:01:30,2017-08-08 00:14:08,accueil,Accès Direct,Ordinateur,NaN,1


## Meta Data

Unamed:0 - Python reading problem

VISITOR_ID - account_number, if not logged in = 0

CONNECTED_SESSION - T/F: logged in or not

ID_SESSION - Single connected session, if VISITOR_ID = 0 it is harder to interpret

SESSION_START_DATE - time user logged in

EVENT_DATE - Triggered by ???

PAGES - Site visited

GLOBAL_SOURCES - Where they came from

DEVICE_TYPE - PC/Table/ect.

DEVICE_MODEL - NaN/iPad/iPhone/

LOADINGS - Refreshed pages

## Dimensions

In [78]:
data_201708.ID_SESSION.value_counts()[:5]

1514    726
1110    635
1283    583
1754    495
3460    492
Name: ID_SESSION, dtype: int64

In [36]:
data_201708['PAGES'].value_counts()[0:5]

accueil                                          230039
mon panier                                       139957
evenement/jeu                                     99125
header::moteur de recherche::page de resultat     77569
ajout au panier                                   76506
Name: PAGES, dtype: int64

## Looking at SESSION 3460's visit

In [81]:
data_201708[data_201708.ID_SESSION == 3460].head()

,Unnamed: 0,VISITOR_ID,CONNECTED_SESSION,ID_SESSION,SESSION_START_DATE,EVENT_DATE,PAGES,GLOBAL_SOURCES,DEVICE_TYPE,DEVICE_MODEL,LOADINGS
25471,25471,0,NON,3460,2017-08-10 08:04:05,2017-08-10 08:04:05,accueil,Accès Direct,Ordinateur,NaN,1
25472,25472,0,NON,3460,2017-08-10 08:04:05,2017-08-10 08:05:23,accueil,Accès Direct,Ordinateur,NaN,1
25473,25473,0,NON,3460,2017-08-10 08:04:05,2017-08-10 08:07:36,accueil,Accès Direct,Ordinateur,NaN,1
25474,25474,0,NON,3460,2017-08-10 08:04:05,2017-08-10 08:08:11,accueil,Accès Direct,Ordinateur,NaN,1
25475,25475,0,NON,3460,2017-08-10 08:04:05,2017-08-10 08:08:30,accueil,Accès Direct,Ordinateur,NaN,1


We have promo codes that indicate if a client used a catalogue or not. 
We assume that the majority of promo code users are catalogue.

If we can establish the characteristics of catalogue users and define the population through a mathematical function. We can then use the same function for other clients to see if they are significantly different from the known catalogue users.

# CLIENTS exploration

In [99]:
clients = pd.read_csv(data_path + 'clients.csv', sep = ';')
clients = clients.loc[:, ~clients.columns.str.contains('^Unnamed')]

In [100]:
clients.head()

,CLIENT_NUMBER,CREATION_DATE,GENDER,BIRTH_YEAR,ZIPCODE
0,-2863680653264308222,2002-12-05,Mme,1955.0,68440
1,6104579344417272025,2002-12-05,Mme,NaN,24100
2,7514073921117581737,2002-12-05,Mme,1945.0,39240
3,-8702883778531324523,2002-12-05,Mme,NaN,42290
4,6338712688710833090,2003-10-06,Mme,1934.0,13006


In [101]:
clients.GENDER.value_counts()

Mme              2714740
M                 286650
Mle               119070
Ste                 2142
MME                 1846
Non Renseigne        805
SOC                  545
ARU                    1
M.                     1
Name: GENDER, dtype: int64

In [102]:
clients.BIRTH_YEAR.value_counts().head()

1950.0    45857
1947.0    44576
1948.0    44021
1946.0    43073
1949.0    42527
Name: BIRTH_YEAR, dtype: int64

In [105]:
clients.ZIPCODE.value_counts().head()

75015    19140
75017    12012
75014    10872
92100    10516
75016    10409
Name: ZIPCODE, dtype: int64

# CMD exploration

In [66]:
cmd = pd.read_csv(data_path + 'CMD.csv', sep = ';')
cmd = cmd[cmd['ORDER_DATE'] >= '2017-08-01']

In [112]:
cmd[cmd['CVIC'] == True]['CLIENT_NUMBER'].value_counts()[:5]

-7419682324447063482    1872
-8029536810428513286    1801
-954373211562525170     1588
 6337010052583634555    1502
 5852995387889251410     972
Name: CLIENT_NUMBER, dtype: int64

In [113]:
cmd

,Unnamed: 0,CLIENT_NUMBER,ORDER_NUMBER,ORDER_DATE,ORDER_CHANNEL,PRE_TAX_AMOUNT,CVIC
1362250,1362250,5435290416568822262,3819077,2017-09-25,Internet,150.42,False
1362834,1362834,-5296242060213987227,3819700,2017-10-10,Internet,160.98,False
1364217,1364217,-8180903312463751979,3821163,2017-12-08,Internet,130.76,False
1364295,1364295,-5670522684245955297,3821248,2017-11-13,Telephone,166.21,True
1364964,1364964,-488144592496065622,3821979,2017-11-27,Internet,76.60,False
1364990,1364990,-8850319371092144317,3822007,2017-11-27,Telephone,143.33,False
1366807,1366807,-4357189758708045171,3823970,2018-01-07,Internet,236.41,False
1369690,1369690,-6217137733543822185,3827201,2018-02-22,Internet,168.82,False
5521433,5521433,-1034782245661405853,9662336,2017-10-03,Magasin,342.00,False
5521633,5521633,-1034782245661405853,9662552,2017-10-03,Magasin,517.66,False
